In [1]:
import sys
sys.path.append('../src')

In [2]:
import ipywidgets as widgets
import pandas as pd
from ipywidgets import interact, interactive, fixed, interact_manual
from pathlib import Path

from covid import daily_snapshot
import population as pop
#import requests

In [3]:
pd.set_option('max_rows', 300) 

In [4]:
def county_populations_for(state: str) -> pd.DataFrame:
    data = Path('../data/counties')
    state_pop_path = data / Path(state.replace(' ', '').lower() + '.csv')
    state_pop = pd.read_csv(state_pop_path)
    return state_pop

In [5]:
def f(state, date, sort_by, ascending):
    daily = daily_snapshot(datetime.strftime(date, '%m-%d-%Y'))
    state_covid = pd.read_csv('../data/us-states-only.csv')
    county_pop = county_populations_for(state)
    state_with_population = pd.merge(state_covid[state_columns], county_pop,
         how='outer',
         left_on='Admin2', right_on='county')

    state_with_population['fraction_confirmed'] = state_with_population['Confirmed'] / state_with_population['pop2019']   * 1000.0
    state_with_population['deaths']             = state_with_population['Deaths']    / state_with_population['pop2019']   * 1000.0
    state_with_population['death_rate?']        = state_with_population['Deaths']    / state_with_population['Confirmed'] * 1000.0

    # sort_by = input("column to sort Confirmed, Deaths, county, pop2019, fraction_confirmed, deaths, death_rate?")
    print(state_with_population.sort_values(by=sort_by, ascending=False))


In [6]:
states = list(pd.read_csv('../data/us-states-only.csv').state)

state = widgets.Select(
    options=states,
    #value='Alabama',
    rows=10,
    description='State:',
    disabled=False
    )

from datetime import datetime
date = widgets.DatePicker(description='Date',
                          value=datetime.today(),
                          disabled=False
                          )

sort_by = widgets.Select(
    options=['Confirmed', 
             'Deaths', 
             'county', 
             'pop2019', 
             'fraction_confirmed', 
             'deaths', 
             'death_rate?'],
    value='fraction_confirmed',
    rows=7,
    description='sort by:',
    disabled=False
    )

ascending = widgets.Checkbox(
    value=True,
    description='Sort Ascending',
    disabled=False,
    indent=False
)

ui = widgets.HBox([state, date, sort_by, ascending])



out = widgets.interactive_output(f, {'state': state, 
                                     'date': date, 
                                     'sort_by': sort_by,
                                     'ascending': ascending})

display(ui, out)

ValueError: No tables found

Output()